# Plots by Sebastian
## Plotting genres part of movies pr. year

### Importing libraries

In [28]:
import matplotlib.pyplot as plt 
from data_gen import gen_df
import numpy as np
import ipywidgets as widgets
import requests
from bs4 import BeautifulSoup as bs

### Generating dataframe from data-file

In [34]:
filename = 'imdb.csv'

df = gen_df(filename)
df['year'] = df['year'].astype(int)

,index,title,imdbRating,ratingCount,duration,year,nrOfWins,nrOfNominations,nrOfPhotos,nrOfNewsArticles,...,News,RealityTV,Romance,SciFi,Short,Sport,TalkShow,Thriller,War,Western
0,2200,The Penalty (1920),7.6,1095.0,1.500000,1920,0.0,0.0,2.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,502,MÌ_dchenlos (1920),8.1,3134.0,2.416667,1920,0.0,0.0,18.0,6.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12832,Irrwege einer Ehe (1920),8.2,1042.0,1.500000,1920,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12148,Das Cabinet des Dr. Caligari (1920),8.1,29379.0,1.300000,1920,0.0,0.0,40.0,122.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12368,Buster Keaton als StrÌ_fling (1920),7.1,1474.0,0.333333,1920,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### Plotting genres as part of total movies at the year
These are generated as:
$$
\text{Genre share} = \frac{\text{Total movies in genre in year}}{\text{Total movies in year}}
$$

In [30]:
# Generate genre list
genre_list = list(df)[12:40]

In [31]:
def _plot_1(df,genre):
    
    df['count'] = 1
    df['movies_year'] = df.groupby('year')['count'].transform(lambda x: x.sum())  

    # Plot year sum of different genres
    for i in genre:
    #i = genre

        df[f'{i}_year'] = df.groupby('year')[i].transform(lambda x: x.sum())
        df[f'{i}_share'] = df[f'{i}_year']/df['movies_year']

        y_share = df.groupby('year')[f'{i}_share'].first()

        #ax = plt.subplot(2,2,j)
        y_share.plot(kind='line', sharex='col', sharey='row')

        plt.xlabel('year')
        plt.ylabel('share of movies')
        plt.title(i)
        
    #plt.show()
    
def plot_1(df):
    
    widgets.interact(_plot_1,
                    df = widgets.fixed(df),
                    genre = widgets.SelectMultiple(
                        options = genre_list,
                        description = 'Genres',
                        disabled = False))
    
plot_1(df)


interactive(children=(SelectMultiple(description='Genres', options=('Action', 'Adult', 'Adventure', 'Animation…

In [32]:

# Condition on minimum number of ratings
I = df['ratingCount'] >= 2000

top = df.sort_values('imdbRating', ascending=False)[I]
bottom = df.sort_values('imdbRating', ascending=True)[I]

# Get English titles 
top_id = top.tid.tolist()[0:10]
bottom_id = bottom.tid.tolist()[0:10]
id_list = top_id + bottom_id

eng_names = []

for i,id in enumerate(id_list):
    print(id)
    test = requests.get('https://www.imdb.com/title/' + id)
    soup = bs(test.text,'html.parser')
    if soup.find('div',class_='originalTitle') != None:
        eng_names.append(soup.find('div',class_='originalTitle').text)
    else:
        eng_names.append(soup.find('h1').text)
    print(i)

#test = df.merge(df_merge,how='right',on='tid')
#print(test.head())



c:\python\python37-32\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
c:\python\python37-32\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


AttributeError: 'DataFrame' object has no attribute 'tid'